In [1]:
# Dependencies to Visualize the model
%matplotlib inline
from IPython.display import display, SVG
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

In [2]:
# Filepaths, numpy, and Tensorflow
import os
import numpy as np
import tensorflow as tf

In [3]:
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from keras.datasets import mnist
from keras.preprocessing import image

Using TensorFlow backend.


In [5]:
import pandas as pd

trainList = []

f_directories = pd.read_csv('food-101/meta/train.txt', header=None)
fList = f_directories[0].tolist()

for item in fList:
    trainList.append(os.path.join('food-101/image',item + '.jpg'))
    
trainList

trainClasses = [x.split("/")[0] for x in fList]

#image_path = trainList[30067]
#image_size = (224,224)

#img = image.load_img(image_path, target_size=image_size)

#plt.imshow(img)

In [7]:
%%time

import imageio

train_imgs = []

for train_img in trainList:
    img = imageio.imread(train_img)
    train_imgs.append(img)


/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6684672 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3368026112 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8519680 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PI

/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 655360 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 33555456 bytes but only got 0. Skipping tag 59392
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 983040 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 524288 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL

/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 102400 bytes but only got 0. Skipping tag 18432
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2162688 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2424832 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PI

/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 51200 bytes but only got 0. Skipping tag 55552
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 25600 bytes but only got 0. Skipping tag 39680
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 204800 bytes but only got 0. Skipping tag 21504
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 25600 bytes but only got 0. Skipping tag 29440
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packa

/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1024000 bytes but only got 0. Skipping tag 13056
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 25600 bytes but only got 0. Skipping tag 15872
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 3. 
  warnings.warn(str(msg))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 140640256 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/anaconda3/envs/PythonData/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possib

CPU times: user 5min 50s, sys: 39.6 s, total: 6min 30s
Wall time: 7min 59s


In [9]:
X_train, y_train = train_imgs, np.array(trainClasses)

In [ ]:
from sklearn.model_selection import tran_test_split
 as 
X = 
y = 


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("Training Data Info")
print("Training Data Shape:", X_train.shape)
print("Training Data Labels Shape:", y_train.shape)